In [ ]:
# Data mainpulation
import pandas as pd
import numpy as np
# Plot
import matplotlib.pyplot as plt
# API
import googleapiclient.discovery
# Google drive
from google.colab import drive
drive.mount('/content/drive')
# HttpError
from googleapiclient.errors import HttpError

Mounted at /content/drive


In [ ]:
from googleapiclient.errors import HttpError

In [ ]:
df = pd.read_csv("/.../videos_index.csv")

In [ ]:
df.head(30)

,title,video_id,description,comment_count,view_count,published_at,channel_subscriber_count,subtitles_text,cleaned_subtitles
0,Is inequality inevitable?,rEnf_CFoyv0,Explore how economic inequality can be measure...,2246,995686.0,2022-10-11T15:01:01Z,19800000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,"In South Africa, one of the most unequal count..."
1,How Wealth Inequality Spiraled Out of Control ...,wOI8RuhW7q0,Here it is: The full story of wealth inequalit...,5955,973519.0,2021-11-03T19:01:04Z,630000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,elon musk's wealth has surpassed 200 billion...
2,Wealth Gap: Last Week Tonight with John Oliver...,LfgSEwjAeno,John Oliver discusses America's growing wealth...,8656,17937666.0,2014-07-14T06:30:01Z,9440000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,our main story tonight is income inequality ...
3,A Look At Income Inequality In The United Stat...,qc7g6Uhi1i4,Income inequality has always existed in the U....,209,130275.0,2020-02-20T21:22:33Z,1330000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,[Music] income inequality has always existed...
4,Thomas Sowell on the Myths of Economic Inequality,mS5WYp5xmvI,"Recorded on November 15, 2018 Thomas Sowell di...",2153,7832958.0,2018-12-03T15:57:22Z,902000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,- It happens to me all the time and it happene...
5,Is Wealth Inequality As Bad As We Are Told? De...,tjQzpt8LeXI,Sign up for my newsletter https://compoundedda...,2075,271601.0,2021-07-23T15:28:17Z,1050000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,The world today is the most uneven it has been...
6,Wealth Inequality in America Has Never Been Wo...,EdqxBNgnmxU,What Wealth Inequality in America looks like. ...,965,230445.0,2023-09-14T23:30:05Z,1250000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,most people have no clue about how severe we...
7,How economic inequality harms societies | Rich...,cZ7LzE3u7Bw,http://www.ted.com We feel instinctively that ...,2547,1120608.0,2011-10-24T18:24:54Z,24200000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,You all know the truth of what I'm going to sa...
8,Why The Inequality Gap Is Growing Between Rich...,41y4c1Oi5Uo,"For forty years, the U.S.-led global economy h...",3038,805400.0,2019-05-29T16:07:11Z,3410000,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,"For 40 years, the U.S. led global economy has ..."
9,Wealth Inequality in America,QPKKQnijnsM,Infographics on the distribution of wealth in ...,65505,25050349.0,2012-11-20T21:48:08Z,25300,WEBVTT\nKind: captions\nLanguage: en\n\n00:00:...,there's a chart I saw recently that I can't ...


Due to the daily 11,000 quota limit, I collected the videos with more comments separately to save time in data collection. Comments are saved separately for each video.

In [ ]:
# Set up Youtube API
api_service_name="youtube"
api_version ="v3"
DEVELOPER_KEY="..."
youtube = googleapiclient.discovery.build(api_service_name,api_version,developerKey=DEVELOPER_KEY)
# Prepare a list to store comments data
comments_data = []
def get_replies(parent_id):
    replies = []
    response = youtube.comments().list(
        part="snippet",
        parentId=parent_id,
        maxResults=100  # maxlimit is 100
    ).execute()

    while response:
        for item in response['items']:
            reply = item['snippet']
            replies.append([
                reply['publishedAt'],
                reply['updatedAt'],
                reply['likeCount'],
                reply['textDisplay'],
                parent_id,  # Include the topLevelCommentId for replies
                True  # mark this as a reply
            ])

        if 'nextPageToken' in response:
            response = youtube.comments().list(
                part="snippet",
                parentId=parent_id,
                maxResults=100,
                pageToken=response['nextPageToken']
            ).execute()
        else:
            break
    return replies

# path
file_path_prefix = '/content/drive/My Drive/Comments/comment_'

for index, video_id in enumerate(df['video_id'][46:], start=47):  # Due to the first index 0, start +1
    comments_data = []
    try:
        response = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            maxResults=100  # maxlimit is 100
        ).execute()

        while response:
            for item in response['items']:
                topLevelComment = item['snippet']['topLevelComment']
                topLevelCommentId = topLevelComment['id']
                topLevelCommentSnippet = topLevelComment['snippet']
                comments_data.append([
                    topLevelCommentSnippet['publishedAt'],
                    topLevelCommentSnippet['updatedAt'],
                    topLevelCommentSnippet['likeCount'],
                    topLevelCommentSnippet['textDisplay'],
                    topLevelCommentId,
                    False  # This is not a reply
                ])

                # Get and append replies
                comments_data.extend(get_replies(topLevelCommentId))

            if 'nextPageToken' in response:
                response = youtube.commentThreads().list(
                    part="snippet,replies",
                    videoId=video_id,
                    maxResults=100,
                    pageToken=response['nextPageToken']
                ).execute()
            else:
                break

        # DataFrame
        df_comments = pd.DataFrame(comments_data, columns=['publishedAt', 'updatedAt', 'likeCount', 'textDisplay', 'commentId', 'is_reply']).drop_duplicates()

        # Save
        file_path = f"{file_path_prefix}{index}.csv"
        df_comments.to_csv(file_path, index=False)
        print(f"Saved comments of video_id {video_id} to {file_path}")
    except HttpError as error:
        print(f"An HTTP error {error.resp.status} occurred for video_id {video_id}: {error.content}")

Saved comments of video_id hNDgcjVGHIw to /content/drive/My Drive/Comments/comment_47.csv
Saved comments of video_id -EVh1tZMIOA to /content/drive/My Drive/Comments/comment_48.csv
Saved comments of video_id JKsHhXwqDqM to /content/drive/My Drive/Comments/comment_49.csv
Saved comments of video_id 12cy_q-pzrA to /content/drive/My Drive/Comments/comment_50.csv
Saved comments of video_id _61OK651AsQ to /content/drive/My Drive/Comments/comment_51.csv
Saved comments of video_id vS7-CTiSZHc to /content/drive/My Drive/Comments/comment_52.csv
Saved comments of video_id -xOLaf4U8d8 to /content/drive/My Drive/Comments/comment_53.csv
Saved comments of video_id PQJboCc-kLw to /content/drive/My Drive/Comments/comment_54.csv
Saved comments of video_id 7fgtE-AJs-A to /content/drive/My Drive/Comments/comment_55.csv
Saved comments of video_id 5CFKEnOh8R0 to /content/drive/My Drive/Comments/comment_56.csv


KeyboardInterrupt: 

The block below is used to separately collect videos with a large comment count.

In [ ]:
# Set up Youtube API
api_service_name="youtube"
api_version ="v3"
DEVELOPER_KEY="..."
youtube = googleapiclient.discovery.build(api_service_name,api_version,developerKey=DEVELOPER_KEY)
# Prepare a list to store comments data
comments_data = []

# Function to get replies for a given comment， I use the endpoint comments().list() to get replies
def get_replies(parent_id):
    replies = []
    response = youtube.comments().list(
        part="snippet",
        parentId=parent_id,
        maxResults=100  # maxlimit is 100
    ).execute()

    while response:
        for item in response['items']:
            reply = item['snippet']
            replies.append([
                reply['publishedAt'],
                reply['updatedAt'],
                reply['likeCount'],
                reply['textDisplay'],
                parent_id,  # Include the topLevelCommentId for replies
                True  # mark this as a reply
            ])

        if 'nextPageToken' in response:
            response = youtube.comments().list(
                part="snippet",
                parentId=parent_id,
                maxResults=100,
                pageToken=response['nextPageToken']
            ).execute()
        else:
            break
    return replies

# Crawling top-level comments by using the endpoint commentThreads().list()
response = youtube.commentThreads().list(
    part="snippet,replies",
    videoId="QPKKQnijnsM",
    maxResults=100  # maxlimit is 100
).execute()

while response:
    for item in response['items']:
        topLevelComment = item['snippet']['topLevelComment']
        topLevelCommentId = topLevelComment['id']
        topLevelCommentSnippet = topLevelComment['snippet']
        comments_data.append([
            topLevelCommentSnippet['publishedAt'],
            topLevelCommentSnippet['updatedAt'],
            topLevelCommentSnippet['likeCount'],
            topLevelCommentSnippet['textDisplay'],
            topLevelCommentId,  # Include the topLevelCommentId for top-level comments
            False  # This is not a reply
        ])

        # Get and append replies
        comments_data.extend(get_replies(topLevelCommentId))

    # Handle pagination for top-level comments
    if 'nextPageToken' in response:
        response = youtube.commentThreads().list(
            part="snippet,replies",
            videoId="QPKKQnijnsM",
            maxResults=100,
            pageToken=response['nextPageToken']
        ).execute()
    else:
        break

# Create DataFrame
df = pd.DataFrame(comments_data, columns=['publishedAt', 'updatedAt', 'likeCount', 'textDisplay', 'commentId', 'is_reply'])

In [ ]:
file_path = '/content/drive/My Drive/Comments/comment_10.csv'
df.to_csv(file_path, index=False)

The block below is used to check whether comments are correct collected.

In [ ]:
len(df)

65507

In [ ]:
df.head(10)

,publishedAt,updatedAt,likeCount,textDisplay,commentId,is_reply
0,2024-04-11T12:55:09Z,2024-04-11T12:55:09Z,0,America is fueled on selfishness and greed it ...,UgwiEC9qJ1YT2S4QXm54AaABAg,False
1,2024-04-08T10:31:10Z,2024-04-08T10:31:10Z,0,Can’t imagine how fucked we are in 2024. The r...,Ugws976dtCEBS3PT-TF4AaABAg,False
2,2024-04-08T13:46:39Z,2024-04-08T13:46:39Z,0,"The way I see it, after years of studying. The...",Ugws976dtCEBS3PT-TF4AaABAg,True
3,2024-04-08T02:22:56Z,2024-04-08T02:22:56Z,0,All lies,UgzYsr5jj_Gy7Ol3pUp4AaABAg,False
4,2024-04-07T18:29:35Z,2024-04-07T18:29:35Z,0,Shouldn&#39;t the chart go negative since many...,Ugy3GFU6aMpEDFPG7gh4AaABAg,False
5,2024-04-07T17:12:15Z,2024-04-07T17:12:15Z,0,&quot;we need to encourage people to work and ...,UgzWhKKnmRKiLHFdYIp4AaABAg,False
6,2024-04-05T02:00:12Z,2024-04-05T02:00:45Z,0,Never forget what you see here. <br>I would L...,UgxVTye_VITsO2RFTeJ4AaABAg,False
7,2024-04-05T01:47:46Z,2024-04-05T01:47:46Z,0,2024 checking in. It’s getting rough out here....,UgxZrcqoEr1mus_YCNh4AaABAg,False
8,2024-04-04T19:31:13Z,2024-04-04T19:31:13Z,0,I think I will go extinct,UgzEA6j0sW7pJExKYGh4AaABAg,False
9,2024-04-03T18:31:34Z,2024-04-03T18:31:34Z,1,They need to remake this video with 2024 stats...,Ugx2gW9WM81PxSwkja54AaABAg,False


Here is a check for whether comments from the correct video.

In [ ]:
# Random Check
from googleapiclient.discovery import build
api_key = "..."
youtube = build('youtube', 'v3', developerKey=api_key)

# Comment ID
comment_id = "UgwY-iKNJa4632V_PPl4AaABAg"

# End point
comment_thread = youtube.commentThreads().list(
    part="snippet",
    id=comment_id
).execute()

# Get url
if 'items' in comment_thread:
    video_id = comment_thread['items'][0]['snippet']['videoId']
    comment_id = comment_thread['items'][0]['snippet']['topLevelComment']['id']
    comment_url = f"https://www.youtube.com/watch?v={video_id}&lc={comment_id}"
    print("Comment URL:", comment_url)
else:
    print("No items found in comment_thread.")

Comment URL: https://www.youtube.com/watch?v=qONOe6bj4gs&lc=UgwY-iKNJa4632V_PPl4AaABAg
